In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Project').getOrCreate()

In [0]:
df = spark.read.csv('/FileStore/tables/hack_data.csv',inferSchema=True,header=True)

In [0]:
df.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)

In [0]:
for item in df.head(1)[0]:
  print(item)

8.0
391.09
1
2.96
7.0
Slovenia
72.37

In [0]:
df.summary().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
summary|Session_Connection_Time| Bytes Transferred| Kali_Trace_Used|Servers_Corrupted| Pages_Corrupted| Location| WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
 count| 334| 334| 334| 334| 334| 334| 334|
 mean| 30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413| null|57.342395209580864|
 stddev| 14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697| 3.06352633036022| null| 13.41106336843464|
 min| 1.0| 10.0| 0| 1.0| 6.0|Afghanistan| 40.0|
 25%| 18.0| 372.05| 0| 3.12| 8.0| null| 44.12|
 50%| 31.0| 600.84| 1| 5.25| 9.0| null| 48.6|
 75%| 42.0| 844.01| 1| 7.4| 14.0| null| 70.58|
 max| 60.0| 1330.5| 1| 10.0| 15.0| Zimbabwe| 75.0|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+

In [0]:
from pyspark.sql import functions as F

from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------+----------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------+----------------+
 0| 0| 0| 0| 0| 0| 0|
+-----------------------+-----------------+---------------+-----------------+---------------+--------+----------------+

In [0]:
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import VectorAssembler

In [0]:
df.columns

Out[25]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [0]:
assembler = VectorAssembler(inputCols=[
 'Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']
                           ,outputCol='features')

In [0]:
df_final = assembler.transform(df)

In [0]:
df_final.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
from pyspark.ml.feature import StandardScaler

In [0]:
scaler = StandardScaler(inputCol='features',outputCol='features_scaled')

In [0]:
scaler_model = scaler.fit(df_final)

In [0]:
scaler_model_transform = scaler_model.transform(df_final)

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
kmeans2 = KMeans(featuresCol='features_scaled',k=2)
kmeans3 = KMeans(featuresCol='features_scaled',k=3)

In [0]:
kmeans2_model = kmeans2.fit(scaler_model_transform)
kmeans3_model = kmeans3.fit(scaler_model_transform)

In [0]:
kmeans3_model.transform(scaler_model_transform).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 83|
 2| 84|
 0| 167|
+----------+-----+

In [0]:
kmeans2_model.transform(scaler_model_transform).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+